# Iris CLASSIFIER
### Let's build a quantum variational algorithm to classify the Iris flowers

In [3]:
# First, we import the libaries
import sys
import random

sys.path.append('../')
import quforge.quforge as qf
from quforge.quforge import State as State

In [4]:
#Load the dataset
from sklearn import datasets

iris = datasets.load_iris()
data = iris['data']
labels = iris['target']

ids = random.sample(range(len(data)), len(data))
ids_train = ids[0:int(0.8*len(data))]
ids_test = ids[int(0.8*len(data)):len(data)]

In [5]:
#Build the model

class Circuit(qf.Module):
    def __init__(self, D, N):
        super(Circuit, self).__init__()

        self.D = D
        self.init = qf.HGate(D=D, index=range(N))
        self.encoder = qf.RZGate(D=D, index=range(N))

        self.layers = qf.Sequential(
            qf.RXGate(D=D, j=0, k=1, index=range(N)),
            qf.RXGate(D=D, j=1, k=2, index=range(N)),
            qf.RXGate(D=D, j=0, k=2, index=range(N)),
            qf.RYGate(D=D, j=0, k=1, index=range(N)),
            qf.RYGate(D=D, j=1, k=2, index=range(N)),
            qf.RYGate(D=D, j=0, k=2, index=range(N)),
            qf.RZGate(D=D, j=1, index=range(N)),
            qf.RZGate(D=D, j=2, index=range(N)),
            qf.CNOT(D=D, N=N, control=0, target=1),
            qf.CNOT(D=D, N=N, control=0, target=2),
            qf.CNOT(D=D, N=N, control=0, target=3),
            qf.RXGate(D=D, j=0, k=1, index=range(N)),
            qf.RXGate(D=D, j=1, k=2, index=range(N)),
            qf.RXGate(D=D, j=0, k=2, index=range(N)),
            qf.RYGate(D=D, j=0, k=1, index=range(N)),
            qf.RYGate(D=D, j=1, k=2, index=range(N)),
            qf.RYGate(D=D, j=0, k=2, index=range(N)),
            qf.RZGate(D=D, j=1, index=range(N)),
            qf.RZGate(D=D, j=2, index=range(N)),
        )

    def forward(self, x):
        y = State('0-0-0-0', D=self.D, device=device)
        y = self.init(y)
        y = self.encoder(y, param=x)
        y = self.layers(y)

        return y

In [6]:
#Instantiate model and optimizer
D = 3 #dimension of the qudit
device = 'cuda'

model = Circuit(D=D, N=4).to(device)
optimizer = qf.optim.Adam(model.parameters(), lr=0.001)

In [7]:
#Prepare the target states
targets = [State('1-0-0', D=D, device=device).flatten(), State('0-1-0', D=D, device=device).flatten(), State('0-0-1', D=D, device=device).flatten()]
targets_arg = [qf.argmax(abs(targets[0])), qf.argmax(abs(targets[1])), qf.argmax(abs(targets[2]))]

In [ ]:
for epoch in range(1):
    acc_train = 0.0
    acc_test = 0.0
    
    for k in ids_train:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], D=D)
        
        loss = qf.mean(abs(targets[label] - m))
        loss_train += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_train += 1

    for k in ids_test:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], D=D)

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_test += 1
    
    acc_train = acc_train/len(ids_train)
    acc_test = acc_test/len(ids_test)

    print(epoch, acc_train, acc_test)